TNT




In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install einops
%pip install wandb

     |████████████████████████████████| 1.8MB 14.0MB/s 
     |████████████████████████████████| 163kB 62.5MB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 133kB 51.4MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=f83ea05bdd367acdb637e749a8c1bfcf2cc3c64f33a3ab357abc482dc14f899f
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=e7ea82b1c6507208594230f037d57c21c8096c0abb3b13b7a5975b9bcbb7d8ba
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [4]:
%cd drive/MyDrive/Colab Notebooks/ViT

/content/drive/MyDrive/Colab Notebooks/ViT


In [5]:
from model.TNT import TNT
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [6]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [7]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 64,
    "patch_size" : 16,
    "num_classes" : 10,
    "dim" : 128,
    "dim_head" : 64,
    "depth" : 12,
    "heads" : 8,
    "dropout" : 0.1,
    }

# Training settings
import wandb
wandb.init(config=config, project="Cifar10_ViT", entity='NoguNogu',name='TNT')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
trainloader, testloader = set_transforms(
    config['image_size'],
    config['batch_size'],
    name='Cifar10')._set_transforms()

In [10]:
model = TNT(
    image_size = config['image_size'],
    patch_dim = config['dim'],
    pixel_dim = 24,
    patch_size = config['patch_size'],
    pixel_size = 4,
    depth= config['depth'],
    num_classes= config['num_classes'],
    heads= config['heads'],
    dim_head = config['dim_head'],
    ff_dropout = config['dropout'],
    attn_dropout = config['dropout'],
    ).to(device)

In [11]:
import torchsummary
torchsummary.summary(model, (3, config['image_size'], config['image_size']))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Rearrange-1            [-1, 3, 16, 16]               0
            Unfold-2               [-1, 48, 16]               0
         Rearrange-3               [-1, 16, 48]               0
            Linear-4               [-1, 16, 24]           1,176
         LayerNorm-5               [-1, 16, 24]              48
            Linear-6             [-1, 16, 1536]          36,864
            Linear-7               [-1, 16, 24]          12,312
           Dropout-8               [-1, 16, 24]               0
         Attention-9               [-1, 16, 24]               0
          PreNorm-10               [-1, 16, 24]               0
        LayerNorm-11               [-1, 16, 24]              48
           Linear-12               [-1, 16, 96]           2,400
             GELU-13               [-1, 16, 96]               0
          Dropout-14               [-1,





```
# tnt = TNT(
    image_size = 256,       # size of image
    patch_dim = 512,        # dimension of patch token
    pixel_dim = 24,         # dimension of pixel token
    patch_size = 16,        # patch size
    pixel_size = 4,         # pixel size
    depth = 6,              # depth
    num_classes = 1000,     # output number of classes
    attn_dropout = 0.1,     # attention dropout
    ff_dropout = 0.1        # feedforward dropout
```






In [12]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [13]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 2.0661 - acc: 0.2294 - val_loss : 1.9306 - val_acc: 0.2907




Epoch : 2 - loss : 1.8879 - acc: 0.3122 - val_loss : 1.8277 - val_acc: 0.3439




Epoch : 3 - loss : 1.7913 - acc: 0.3499 - val_loss : 1.7417 - val_acc: 0.3729




Epoch : 4 - loss : 1.7227 - acc: 0.3772 - val_loss : 1.6995 - val_acc: 0.3893




Epoch : 5 - loss : 1.6731 - acc: 0.3952 - val_loss : 1.6476 - val_acc: 0.4085



...


Epoch : 16 - loss : 1.4621 - acc: 0.4753 - val_loss : 1.4748 - val_acc: 0.4698




Epoch : 17 - loss : 1.4477 - acc: 0.4799 - val_loss : 1.4631 - val_acc: 0.4721




Epoch : 18 - loss : 1.4370 - acc: 0.4861 - val_loss : 1.4645 - val_acc: 0.4729




Epoch : 19 - loss : 1.4307 - acc: 0.4883 - val_loss : 1.4412 - val_acc: 0.4802




Epoch : 20 - loss : 1.4122 - acc: 0.4933 - val_loss : 1.4363 - val_acc: 0.4877



In [14]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon May 10 03:31:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    51W / 300W |   3795MiB / 16160MiB |     41%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------